In [1]:
from KGNET import KGNET

/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kgnet=KGNET(KG_endpointUrl='http://206.12.98.118:8890/sparql',KG_NamedGraph_IRI='https://dblp2022.org',KG_Prefix='dblp2022')

## visualize graph schema

In [ ]:
types_df=kgnet.getKGNodeEdgeTypes()
g=kgnet.visualizeKG(types_df,Notebook=True,width="100%",height="1000px")
g.show('nx.html')

In [ ]:
types_df

## Training Pipeline

In [ ]:
TargetEdge="https://dblp.org/rdf/schema#authoredBy"

In [ ]:

taskId,modelId,trainMetadata=kgnet.train_GML( operatorType=KGNET.GML_Operator_Types.LinkPrediction,
                                                       targetEdge=TargetEdge,
                                                       GNNMethod=KGNET.GNN_Methods.MorsE)

### Query Task Info from KGMeta

In [ ]:
kgnet.KGMeta_Governer.getGMLTaskByID(taskId)

### Query Model Info from KGMeta

In [ ]:
kgnet.KGMeta_Governer.getGMLModelByID(modelId)

## Inference Pipeline
write the SPARQL-ML inference query then execute it. KGNet will choose the model that match your task.

In [ ]:
query = """
                    prefix dblp2022:<https://dblp.org/rdf/schema#>
                    prefix kgnet:<https://kgnet/>
                    select ?publication ?author                    
                    from <https://dblp2022.org>
                    where {
                    ?publication a dblp2022:Publication.
                    ?publication ?LinkPredictor ?author.
                    ?LinkPredictor  a <kgnet:types/LinkPredictor>.
                    ?LinkPredictor  <kgnet:targetEdge> \""""+TargetEdge+"""\" .
                    ?LinkPredictor <kgnet:GNNMethod> "MorsE" .
                    ?LinkPredictor <kgnet:topK> 3 .
                    }
                    limit 300
                    offset 0
        """

In [ ]:
res_df,inferenceQueryMetadata=kgnet.executeSPARQLMLInferenceQuery(query)

In [ ]:
res_df

In [ ]:
#for key in inferenceQueryMetadata:
#    print("####################"+key+"#######################")
#    print(inferenceQueryMetadata[key])